In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls -al '/content/drive/My Drive/perf_eng_HW2/'

total 57
-rw------- 1 root root   792 May 24 08:03 alias.hpp
-rw------- 1 root root 13381 May 24 08:05 CMakeCache.txt
drwx------ 2 root root  4096 May 24 08:04 CMakeFiles
-rw------- 1 root root  1518 May 24 08:05 cmake_install.cmake
-rw------- 1 root root   118 May 24 08:03 CMakeLists.txt
drwx------ 2 root root  4096 May 24 08:02 data
-rw------- 1 root root 20088 May 24 08:09 HW2
-rw------- 1 root root   342 May 24 08:03 main.cpp
-rw------- 1 root root  5531 May 24 08:09 Makefile
-rw------- 1 root root   273 May 24 08:08 test.hpp
-rw------- 1 root root   819 May 24 08:06 test.ipp
-rw------- 1 root root  3574 May 24 08:03 utils.cpp
-rw------- 1 root root   565 May 24 08:03 utils.hpp


In [3]:
!cd '/content/drive/My Drive/perf_eng_HW2/'  && cmake . && make clean && make HW2 && ./HW2

-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/My Drive/perf_eng_HW2
[-33%] Building CXX object CMakeFiles/HW2.dir/main.cpp.o
[  0%] Building CXX object CMakeFiles/HW2.dir/utils.cpp.o
[ 33%] Linking CXX executable HW2
[ 33%] Built target HW2
Function 	 Time 
red:    167214720
green:  167213850
blue:   167213715
sum(CPU)	5	ms
-----------------
red:    0
green:  0
blue:   0
min(CPU)	3	ms
-----------------
red:    128
green:  26
blue:   147
integral image(CPU)	14	ms


In [4]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [5]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gi86_btr
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gi86_btr
  Stored in directory: /tmp/pip-ephem-wheel-cache-sih65k1u/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


**Task1**

In [7]:
%%cu

//
// Created by Yevhen Pozdniakov on 5/2/19.
//

#include <stdio.h>
#include <iostream>
#include <chrono>
#include <time.h>
#include <fstream>
#include <stdint.h>

#include <math.h>

using res_t = uint64_t;

constexpr int num_iterations = 1;

constexpr int block_size = 1024;

enum colors : uint8_t
{
    red = 1,
    green = 2,
    blue = 3,
    last
};

struct img_t
{
    int width;
    int height;
    uint8_t * data;
};

#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error at runtime: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

img_t read_bmp(const char* filename)
{
    FILE* f = fopen(filename, "rb");

    if(f == NULL)
    {
        throw "Argument Exception";
    }

    unsigned char info[54];
    fread(info, sizeof(unsigned char), 54, f); // read the 54-byte header

    img_t img;
    // extract image height and width from header
    img.width = *(int*)&info[18];
    img.height = *(int*)&info[22];

/*
    std::cout << std::endl;
    std::cout << "Name: " << filename << std::endl;
    std::cout << "Width: " << img.width << std::endl;
    std::cout << "Height: " << img.height << std::endl;
*/
    int row_padded = (img.width*3 + 3) & (~3);
    img.data = new uint8_t [row_padded*img.height];
    uint8_t * buf = new uint8_t [row_padded];
    for(auto i = 0u; i < img.height; ++i)
    {
        auto start_idx_r = i*img.width;
        auto start_idx_g = i*img.width + img.height*img.width;
        auto start_idx_b = i*img.width + img.height*img.width*2;
        fread(buf, sizeof(uint8_t), row_padded, f);
        auto k = 0u;
        for(auto j = 0u; j < row_padded; j += 3)
        {
            // Convert (B, G, R) to (R, G, B)
            img.data[start_idx_r + k] = buf[j+2];
            img.data[start_idx_g + k] = buf[j+1];
            img.data[start_idx_b + k] = buf[j];
            ++k;

//            std::cout << "R: "<< (int)img.data[start_idx_r + j] << " G: " << (int)img.data[start_idx_g + j]
//                      << " B: " << (int)img.data[start_idx_b + j]<< std::endl;
        }
    }
    delete [] buf;
    fclose(f);
    return img;
}

__global__ void sum_reduce_simple(unsigned char *g_ivec,  res_t *g_ovec)
{    
    extern __shared__ int sdata[];
    //each thread load s one element from global to shared mem
    unsigned int tid = threadIdx.x;
    unsigned int i =  blockIdx.x * blockDim.x + threadIdx.x;
    sdata[tid] = g_ivec[i];
    __syncthreads();

    // do reduction in shared mem
    for (auto s = 1u; s <  blockDim.x ; s *= 2)
    {
        // && ((tid + s) % 3 != ch_idx)
        if (tid % (2*s) == 0)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    // write result for this block to global mem
    if (tid == 0) g_ovec[blockIdx.x] = sdata[0];
}

__global__ void sum_reduce_simple(res_t *g_ivec,  res_t *g_ovec){
    extern __shared__ int sdata[];

    //each thread load s one element from global to shared mem
    unsigned int tid = threadIdx.x;
    unsigned int i =  blockIdx.x * blockDim.x + threadIdx.x;
    sdata[tid] = g_ivec[i];
    __syncthreads();

    // do reduction in shared mem
    
    for (unsigned int s=1; s <  blockDim.x ; s *= 2)
    {
        if (tid % (2*s) == 0)
        {
            //if ((tid + s) > blockDim.x)
              //printf("!|tid = %d, s = %d, sdata = %d; ", tid, s, sdata[tid+s]);
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    // write result for this block to global mem
    if (tid == 0) g_ovec[blockIdx.x] = sdata[0];
}

res_t sum_chunk(const uint8_t* arr, size_t total_size)
{   
     //ALLOCATE MEM
    unsigned char *d_image;
    
    cudaMalloc(&d_image, total_size);
    cudaCheckErrors("cudaMalloc fail \n");
    
    cudaMemcpy(d_image, arr, total_size*sizeof(uint8_t), cudaMemcpyHostToDevice);
    cudaCheckErrors("Kernel CALL fail \n");   
    
    res_t *d_result;
    cudaMalloc(&d_result, block_size*sizeof(res_t));
    cudaCheckErrors("Memory filling failed \n");

    sum_reduce_simple <<< block_size, block_size, block_size*sizeof(res_t) >>> (d_image, d_result);
    cudaCheckErrors("Kernel sum_reduce_simple CALL fail \n");

    sum_reduce_simple <<< 1, block_size, block_size*sizeof(res_t) >>> (d_result, d_result);
    cudaCheckErrors("Kernel sum_reduce_simple CALL fail \n");

    res_t *h_result = (res_t*)malloc(block_size*sizeof(res_t));
    cudaMemcpy(h_result, d_result, block_size*sizeof(res_t), cudaMemcpyDeviceToHost);
    cudaCheckErrors("Memory copying result fail \n");
    
    res_t res = h_result[0];
    
    //FREE MEM
    cudaFree(d_image);
    cudaFree(d_result);
    cudaCheckErrors("cudaFree fail \n");
    free(h_result);

    return res;
}


res_t sum_of_pixels_in_channel(const img_t img, colors channel)
{
    
    //img_t img = read_bmp("/content/drive/My Drive/perf_eng_HW2/data/img-2.bmp");
    
    //size_t total_size = img.height * img.width * 3;
    
    //printf("h = %d, w = %d. ", rows_num, cols_num);
    auto start_idx = (static_cast<int>(channel) - 1) * img.width*img.height;
    auto end_idx = start_idx + (img.width * img.height);
    res_t sum = 0;
    for(auto k = start_idx; k < end_idx; k+=block_size*block_size)
    {
      size_t rest = (end_idx - k) > block_size*block_size
                      ? block_size*block_size
                      : end_idx - k;
      sum+=sum_chunk(&img.data[k], rest);   
    }

    return sum;
}

void test_task1()
{
    auto img = read_bmp("/content/drive/My Drive/perf_eng_HW2/data/img-2_small.bmp");

    cudaEvent_t start, stop;
    float time;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    for (auto i = 0u; i < num_iterations; ++i)
    {
//        sum_of_pixels_in_channel(img, colors::red);
//        sum_of_pixels_in_channel(img, colors::green);
//        sum_of_pixels_in_channel(img, colors::blue);
        std::cout << "sum(r): " << sum_of_pixels_in_channel(img, colors::red) << ", ";
        std::cout << "sum(g): " << sum_of_pixels_in_channel(img, colors::green) << ", ";
        std::cout << "sum(b): " << sum_of_pixels_in_channel(img, colors::blue) << ", ";
        //std::cout << "total: " << sum_of_pixels_in_channel(img, colors::blue) + sum_of_pixels_in_channel(img, colors::red) + sum_of_pixels_in_channel(img, colors::green) << std::endl;
    }

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time, start, stop);
    printf ("test_task1: %f ms. ", time);

    delete [] img.data;
}

int main()
{    
    test_task1();
   
    return(0);
}

'sum(r): 167214720, sum(g): 167213850, sum(b): 167213715, test_task1: 4.112768 ms. '

Task1(CPU): 17ms
<br>Task1(Cuda): ~11ms

**Task2**

In [8]:
%%cu

//
// Created by Yevhen Pozdniakov on 5/2/19.
//

#include <stdio.h>
#include <iostream>
#include <chrono>
#include <time.h>
#include <fstream>
#include <stdint.h>

#include <math.h>

constexpr int num_iterations = 1;

constexpr int block_size = 1024;

enum colors : uint8_t
{
    red = 1,
    green = 2,
    blue = 3,
    last
};

struct img_t
{
    int width;
    int height;
    uint8_t * data;
};

#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error at runtime: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

  
img_t read_bmp(const char* filename)
{
    FILE* f = fopen(filename, "rb");

    if(f == NULL)
    {
        throw "Argument Exception";
    }

    unsigned char info[54];
    fread(info, sizeof(unsigned char), 54, f); // read the 54-byte header

    img_t img;
    // extract image height and width from header
    img.width = *(int*)&info[18];
    img.height = *(int*)&info[22];

/*
    std::cout << std::endl;
    std::cout << "Name: " << filename << std::endl;
    std::cout << "Width: " << img.width << std::endl;
    std::cout << "Height: " << img.height << std::endl;
*/
    int row_padded = (img.width*3 + 3) & (~3);
    img.data = new uint8_t [row_padded*img.height];
    uint8_t * buf = new uint8_t [row_padded];
    for(auto i = 0u; i < img.height; ++i)
    {
        auto start_idx_r = i*img.width;
        auto start_idx_g = i*img.width + img.height*img.width;
        auto start_idx_b = i*img.width + img.height*img.width*2;
        fread(buf, sizeof(uint8_t), row_padded, f);
        auto k = 0u;
        for(auto j = 0u; j < row_padded; j += 3)
        {
            // Convert (B, G, R) to (R, G, B)
            img.data[start_idx_r + k] = buf[j+2];
            img.data[start_idx_g + k] = buf[j+1];
            img.data[start_idx_b + k] = buf[j];
            ++k;

//            std::cout << "R: "<< (int)img.data[start_idx_r + j] << " G: " << (int)img.data[start_idx_g + j]
//                      << " B: " << (int)img.data[start_idx_b + j]<< std::endl;
        }
    }
    delete [] buf;
    fclose(f);
    return img;
}


__global__ void min_reduce_simple(unsigned char *g_ivec,  unsigned char *g_ovec)
{    
    extern __shared__ int sdata[];
    //each thread load s one element from global to shared mem
    unsigned int tid = threadIdx.x;
    unsigned int i =  blockIdx.x * blockDim.x + threadIdx.x;
    sdata[tid] = g_ivec[i];
    __syncthreads();

    // do reduction in shared mem
    for (auto s = 1u; s <  blockDim.x ; s *= 2)
    {
        // && ((tid + s) % 3 != ch_idx)
        if (tid % (2*s) == 0)
        {
            sdata[tid] = sdata[tid] < sdata[tid + s] ? sdata[tid] : sdata[tid + s];
        }
        __syncthreads();
    }
    
    // write result for this block to global mem
    if (tid == 0) g_ovec[blockIdx.x] = sdata[0];
}


unsigned char chunk(const uint8_t* arr, size_t total_size)
{   
     //ALLOCATE MEM
    unsigned char *d_image;
    
    cudaMalloc(&d_image, total_size);
    cudaCheckErrors("cudaMalloc fail \n");
    
    cudaMemcpy(d_image, arr, total_size*sizeof(uint8_t), cudaMemcpyHostToDevice);
    cudaCheckErrors("Kernel CALL fail \n");   
    
    unsigned char *d_result;
    cudaMalloc(&d_result, block_size*sizeof(unsigned char));
    cudaCheckErrors("Memory filling failed \n");

    min_reduce_simple <<< block_size, block_size, block_size*sizeof(int) >>> (d_image, d_result);
    cudaCheckErrors("Kernel sum_reduce_simple CALL fail \n");

    unsigned char *h_result = (unsigned char*)malloc(block_size*sizeof(unsigned char));
    cudaMemcpy(h_result, d_result, block_size*sizeof(unsigned char), cudaMemcpyDeviceToHost);
    cudaCheckErrors("Memory copying result fail \n");
    
    unsigned char res = h_result[0];
    
    //FREE MEM
    cudaFree(d_image);
    cudaFree(d_result);
    cudaCheckErrors("cudaFree fail \n");
    free(h_result);

    return res;
}


unsigned char min_in_channel(const img_t img, colors channel)
{
    auto start_idx = (static_cast<int>(channel) - 1) * img.width*img.height;
    auto end_idx = start_idx + (img.width * img.height);

    unsigned char min = 255;
    for(auto k = start_idx; k < end_idx; k+=block_size*block_size)
    {
      size_t rest = (end_idx - k) > block_size*block_size
                      ? block_size*block_size
                      : end_idx - k;
      auto cur_min = chunk(&img.data[k], rest);   
      if (cur_min < min)
      {
        min = cur_min;      
      }
    }

    return min;
}


void test_task2()
{
    auto img = read_bmp("/content/drive/My Drive/perf_eng_HW2/data/img-2_small.bmp");

    cudaEvent_t start, stop;
    float time;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    for (auto i = 0u; i < num_iterations; ++i)
    {
        std::cout << "min(r): " << (int)min_in_channel(img, colors::red) << ", ";
        std::cout << "min(g): " << (int)min_in_channel(img, colors::green) << ", ";
        std::cout << "min(b): " << (int)min_in_channel(img, colors::blue) << ", ";
    }

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time, start, stop);
    printf ("test_task2: %f ms. ", time);

    delete [] img.data;
}


int main()
{    
    test_task2();
   
    return(0);
}

'min(r): 0, min(g): 0, min(b): 0, test_task2: 3.839456 ms. '

Task2(CPU): 12ms
<br>Task2(Cuda): ~11ms

**Task3**

In [42]:
%%cu

//
// Created by Yevhen Pozdniakov on 5/2/19.
//

#include <stdio.h>
#include <iostream>
#include <chrono>
#include <time.h>
#include <fstream>
#include <stdint.h>

#include <math.h>

constexpr int num_iterations = 1;

constexpr int block_size = 1024;

enum colors : uint8_t
{
    red = 1,
    green = 2,
    blue = 3,
    last
};

struct img_t
{
    int width;
    int height;
    uint8_t * data;
    uint8_t * r;
    uint8_t * g;
    uint8_t * b;
};

uint8_t * get_channel_data(const img_t& img, colors color)
{
    uint8_t* data = nullptr;

    if (color == colors::red)
        data = img.r;
    else if (color == colors::green)
        data = img.g;
    else
        data = img.b;

    return data;
}

void print_channel(const uint8_t* data, std::size_t width, std::size_t height)
    {
        for (auto i = 0u; i < width*height; ++i)
        {
            if (i%width == 0) std::cout<<std::endl;
            std::cout<<int(data[i])<<" ";
        }
        std::cout<<std::endl;
    }
  
img_t read_bmp(const char* filename)
{
    FILE* f = fopen(filename, "rb");

    if(f == NULL)
    {
        throw "Argument Exception";
    }

    unsigned char info[54];
    fread(info, sizeof(unsigned char), 54, f); // read the 54-byte header

    img_t img;
    // extract image height and width from header
    img.width = *(int*)&info[18];
    img.height = *(int*)&info[22];

//    std::cout << std::endl;
//    std::cout << "Name: " << filename << std::endl;
//    std::cout << "Width: " << img.width << std::endl;
//    std::cout << "Height: " << img.height << std::endl;

    int row_padded = (img.width*3 + 3) & (~3);
    img.data = new uint8_t [row_padded*img.height];
    uint8_t * buf = new uint8_t [row_padded];
    for(auto i = 0u; i < img.height; ++i)
    {
//        auto start_idx = i*img.width*3;
        auto start_idx_r = i*img.width;
        auto start_idx_g = i*img.width + img.height*img.width;
        auto start_idx_b = i*img.width + img.height*img.width*2;

        fread(buf, sizeof(uint8_t), row_padded, f);
        auto k = 0u;
        for(auto j = 0u; j < row_padded; j += 3)
        {
            // Convert (B, G, R) to (R, G, B)
            img.data[start_idx_r + k] = buf[j+2];// == 0 ? 1 : buf[j+2];
            img.data[start_idx_g + k] = buf[j+1];
            img.data[start_idx_b + k] = buf[j];
            ++k;

//            std::cout << "R: "<< (int)img.data[start_idx_r + j] << " G: " << (int)img.data[start_idx_g + j]
//                      << " B: " << (int)img.data[start_idx_b + j]<< std::endl;
        }
    }
    img.r = &img.data[0];
    img.g = &img.data[img.height*img.width];
    img.b = &img.data[2*img.height*img.width];
    
    delete [] buf;
    fclose(f);
    return img;
}

#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error at runtime: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

__global__ void convert_to_integral_cuda(uint8_t *a, uint8_t *b, int rowsTotal, int colsTotal)
{
    // Thread Ids equal to block Ids because the each blocks contains one thread only.
    int col = blockIdx.x;
    int row = blockIdx.y;
    uint8_t temp = 0;

    if(col < colsTotal && row < rowsTotal)
    {
        // The first loop iterates from zero to the Y index of the thread which represents the corresponding element of the output/input array.  
        for(int r = 0; r <= row; ++r)
        {
            // The second loop iterates from zero to the X index of the thread which represents the corresponding element of the output/input array  
            for(int c = 0; c <= col; ++c)
            {
                temp = temp + a[r*colsTotal+c];
            }
        }
    }
    
    __syncthreads();

    //Transfer the final result to the output array
    b[row*colsTotal+col] = temp;
}

unsigned char convert_to_integral(const img_t img, colors color)
{
    uint8_t* data = get_channel_data(img, color);

    //M is number of rows
    //N is number of columns

    int M, N;
    M = img.height;
    N = img.width;
    int total_e = M*N;
    int widthstep = total_e*sizeof(uint8_t);
    
    uint8_t * matrix_a = data;
    uint8_t * matrix_b = (uint8_t *)malloc(widthstep);
    
    /*
    for(int r=0;r<M;r++)
    {
        for(int c=0; c<N;c++)
        {
            //cout<<"Enter Matrix element [ "<<r<<","<<c<<"]";
            //cin>>m_e;
            matrix_a[r*N+c]=r*N+c;
            matrix_b[r*N+c]=0;
        }
    }
    */
    
    //print_channel(matrix_a, img.width, img.height);

    uint8_t * d_matrix_a, * d_matrix_b;
    cudaMalloc(&d_matrix_a, widthstep);
    cudaCheckErrors("cudaMalloc 'd_matrix_a' fail \n");
    cudaMalloc(&d_matrix_b, widthstep);
    cudaCheckErrors("cudaMalloc 'd_matrix_b' fail \n");

    cudaMemcpy(d_matrix_a, matrix_a, widthstep, cudaMemcpyHostToDevice);
    cudaCheckErrors("cudaMemcpy 'matrix_a' fail \n");
    cudaMemcpy(d_matrix_b, matrix_b, widthstep, cudaMemcpyHostToDevice);
    cudaCheckErrors("cudaMemcpy 'matrix_b' fail \n");
    
    //Creating a grid where the number of blocks are equal to the number of pixels or input matrix elements.

    //Each block contains only one thread.

    dim3 grid(N, M);
    convert_to_integral_cuda<<<grid, 1>>>(d_matrix_a, d_matrix_b, M, N);
    //convert_to_integral_cuda<<<grid, 1>>>(d_matrix_a, d_matrix_b, M, N);
    //cudaThreadSynchronize();

    cudaMemcpy(matrix_b, d_matrix_b, widthstep, cudaMemcpyDeviceToHost);
    cudaCheckErrors("cudaMemcpy 'd_matrix_b' fail \n");

    //print_channel(matrix_b, img.width, img.height);
    
    auto res =  matrix_b[img.width*img.height - 1];
    cudaFree(d_matrix_a);
    cudaCheckErrors("cudaFree 'd_matrix_a' fail \n");
    cudaFree(d_matrix_b);
    cudaCheckErrors("cudaFree 'd_matrix_b' fail \n");
    free(matrix_b);

    return res;
}


void test_task3()
{
    auto img = read_bmp("/content/drive/My Drive/perf_eng_HW2/data/img-2_small.bmp");
    /*
    img_t grey_img;
    grey_img.data=new uint8_t[12];
    grey_img.width = 2;
    grey_img.height = 2;
    
    grey_img.data[0] = 5;
    grey_img.data[1] = 2;
    grey_img.data[2] = 5;
    grey_img.data[3] = 2;

    grey_img.data[4] = 3;
    grey_img.data[5] = 6;
    grey_img.data[6] = 3;
    grey_img.data[7] = 6;

    grey_img.data[8] = 5;
    grey_img.data[9] = 2;
    grey_img.data[10] = 5;
    grey_img.data[11] = 2;
    grey_img.r = &grey_img.data[0];
    grey_img.g = &grey_img.data[4];
    grey_img.b = &grey_img.data[8];
   */

    cudaEvent_t start, stop;
    float time;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    for (auto i = 0u; i < num_iterations; ++i)
    {
        //convert_to_integral(grey_img, colors::red);
        std::cout << "r: " << (int)convert_to_integral(img, colors::red) << ", ";
        std::cout << "g: " << (int)convert_to_integral(img, colors::green) << ", ";
        std::cout << "b: " << (int)convert_to_integral(img, colors::blue) << ", ";
    }

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time, start, stop);
    printf ("test_task3: %f ms. ", time);

    delete [] img.data;
}


int main()
{    
    test_task3();
   
    return(0);
}

'r: 128, g: 26, b: 147, test_task3: 12552.872070 ms. '

Task3(CPU): 14ms
<br>Task3(Cuda): 12552ms